# TP: Sales forecasting

The goal of this TP is to build an algorithm to make some sales forecasting model. Forecast the sales for a specific day.

For this you have multiples files containing different informations

The TP is divided into 3 parts:

- Data analysis (understand the data and make some plots)

- Data preprocessing (train / test split + features preprocessing)

- Build & train models:
    - Decision tree
    - Random forest
    - GBDT


BONUS: Prophet model from faceboost (https://facebook.github.io/prophet/docs/quick_start.html#python-api / https://medium.com/illumination/understanding-fb-prophet-a-time-series-forecasting-algorithm-c998bc52ca10)

In [ ]:
!ls drive/MyDrive/datasets_ecoleit/4DEP/tp3/dataset/

ALL_COMMODITIES_data.csv  COTTON_data.csv      NATURAL_GAS_data.csv  SUGAR_data.csv
ALUMINUM_data.csv	  data.csv	       oil.csv		     transactions.csv
COPPER_data.csv		  holidays_events.csv  stores.csv


**data.csv**
-------

The data (train & test), comprising time series of features store_nbr, family, and onpromotion as well as the target sales.

**store_nbr** identifies the store at which the products are sold.
family identifies the type of product sold.

**sales** gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).

**onpromotion** gives the total number of items in a product family that were being promoted at a store at a given date.


**test.csv**
-------

The test data, having the same features as the training data. You will predict the target sales for the dates in this file.
The dates in the test data are for the 15 days after the last date in the training data.

**transactions.csv**
------
Information of the transactions for each store.

It contains the day, the store id and the number of transactions

**stores.csv**
------
Store metadata, including city, state, type, and cluster.

cluster is a grouping of similar stores.

**oil.csv**
------
Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)

**holidays_events.csv**
------

Holidays and Events, with metadata

NOTE: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.

Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).


**Other df added to help you to have better forecasting**
------

ALL_COMMODITIES_data.csv

COTTON_data.csv	oil.csv

ALUMINUM_data.csv

COPPER_data.csv

NATURAL_GAS_data.csv

SUGAR_data.csv

**GOAL**
-----
The Goal of this TP is to forecast the 'sales' column of the data.csv

-> We want to be able to predict the sales value for a specific day


# 1. Data loading and analysis

In [1]:
import os

import pandas as pd

In [2]:
base_path = 'drive/MyDrive/datasets_ecoleit/4DEP/tp3/dataset/'

Load the data

In [3]:
df = pd.read_csv(os.path.join(base_path, 'data.csv'))
df

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


Display the list of unique values for the column store_nbr

In [4]:
df['store_nbr'].unique()

array([ 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,  2, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29,  3, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,  4,
       40, 41, 42, 43, 44, 45, 46, 47, 48, 49,  5, 50, 51, 52, 53, 54,  6,
        7,  8,  9])

As you can notice in the data.csv

We have multiple time the same dates, for instance we have multiple time 2023-01-01 for each store nb and each family (product).

It means our model need to be able to predict for a specific date / store / product family / number of promotions what is going to be the number of sales

Plot the number of promotion by product family

In [5]:
import plotly.express as px

grouped_data = df.groupby('family')['onpromotion'].sum().reset_index()

fig = px.bar(grouped_data, x='family', y='onpromotion', title='Number of Items on Promotion by Family Category')
fig.show()

Plot for each store the number of promotions

In [6]:
grouped_data = df.groupby('store_nbr')['onpromotion'].sum().reset_index()

fig = px.bar(grouped_data, x='store_nbr', y='onpromotion', title='Number of Items on Promotion by Store')
fig.show()

Plot the sales amount per day for a specific store

In [8]:
store_number = 30

specific_store_df = df[df['store_nbr'] == store_number]

daily_sales = specific_store_df.groupby('date')['sales'].sum().reset_index()

# Create a line plot
fig = px.line(daily_sales, x='date', y='sales', title=f'Daily Sales for Store Number {store_number}')
fig.show()

Display the unique values of product family

In [9]:
df['family'].unique()

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

Plot the sales amount per day for a specific product family for all stores

In [10]:
product_name = 'HARDWARE'

specific_store_df = df[df['family'] == product_name]

daily_sales = specific_store_df.groupby('date')['sales'].sum().reset_index()

# Create a line plot
fig = px.line(daily_sales, x='date', y='sales', title=f'Daily Sales for the product {product_name}')
fig.show()

Plot the sales amount for all store and all product for each day

In [14]:
daily_sales = df.groupby('date')['sales'].sum().reset_index()

# Create a line plot
fig = px.line(daily_sales, x='date', y='sales', title=f'Daily Sales amount')
fig.show()

Compute & plot the total number of sales by year

In [15]:
# Convert 'date' to datetime and extract the year
df['year'] = pd.to_datetime(df['date']).dt.year

# Group by 'year' and sum 'sales'
yearly_sales = df.groupby('year')['sales'].sum().reset_index()

# Create a bar plot
fig = px.bar(yearly_sales, x='year', y='sales', title='Total Sales amount by Year')
fig.show()

In [16]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

Plot a graph with the daily sales and daily promotions to be able to compare them

In [17]:
# Convert 'date' to datetime
df['date'] = pd.to_datetime(df['date'])

# Group by 'date' and sum 'sales' and 'onpromotion'
daily_data = df.groupby('date').agg({'sales': 'sum', 'onpromotion': 'sum'}).reset_index()

# Create a line plot with two lines
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add sales line
fig.add_trace(
    go.Scatter(x=daily_data['date'], y=daily_data['sales'], name='Daily Sales'),
    secondary_y=False,
)

# Add onpromotion line
fig.add_trace(
    go.Scatter(x=daily_data['date'], y=daily_data['onpromotion'], name='Daily Promotions'),
    secondary_y=True,
)

# Add figure title
fig.update_layout(title_text="Daily Sales and Promotions")

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="Total Sales", secondary_y=False)
fig.update_yaxes(title_text="Number of Promotions", secondary_y=True)

fig.show()


Load transactions.csv

In [19]:
df_transactions = pd.read_csv(os.path.join(base_path, 'transactions.csv'))
df_transactions

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
...,...,...,...
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932


Display the number of transactions by day for all stores

In [20]:
# Create a line plot

daily_transactions = df_transactions.groupby('date')['transactions'].sum().reset_index()

fig = px.line(daily_transactions, x='date', y='transactions', title=f'Daily number of transactions')
fig.show()

In your previous plot, zoom on a specific period (you can do this with plotly)

Check on internet the day of the week for this specific period

What can you conclude ?

Display the number of daily transactions for a specific store

In [22]:
# Create a line plot

store_nbr = 3

filtered_df_transactions = df_transactions[df_transactions['store_nbr'] == store_nbr]

daily_transactions = filtered_df_transactions.groupby('date')['transactions'].sum().reset_index()

fig = px.line(daily_transactions, x='date', y='transactions', title=f'Daily number of transactions for store {store_nbr}')
fig.show()

Join the data with the train.csv

on date and store number

replace the NaN values by 0

In [23]:
df.shape

(3000888, 7)

In [24]:
df_transactions

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
...,...,...,...
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932


In [25]:
df_transactions['date'] = pd.to_datetime(df_transactions['date'])

In [26]:
df_train_with_transactions = pd.merge(df_transactions, df, on=['date', 'store_nbr'], how='right')
df_train_with_transactions = df_train_with_transactions.fillna(0)
df_train_with_transactions

,date,store_nbr,transactions,id,family,sales,onpromotion,year
0,2013-01-01,1,0.0,0,AUTOMOTIVE,0.000,0,2013
1,2013-01-01,1,0.0,1,BABY CARE,0.000,0,2013
2,2013-01-01,1,0.0,2,BEAUTY,0.000,0,2013
3,2013-01-01,1,0.0,3,BEVERAGES,0.000,0,2013
4,2013-01-01,1,0.0,4,BOOKS,0.000,0,2013
...,...,...,...,...,...,...,...,...
3000883,2017-08-15,9,2155.0,3000883,POULTRY,438.133,0,2017
3000884,2017-08-15,9,2155.0,3000884,PREPARED FOODS,154.553,1,2017
3000885,2017-08-15,9,2155.0,3000885,PRODUCE,2419.729,148,2017
3000886,2017-08-15,9,2155.0,3000886,SCHOOL AND OFFICE SUPPLIES,121.000,8,2017


In [27]:
del df

In [28]:
del df_transactions

Load stores.csv

In [29]:
df_stores = pd.read_csv(os.path.join(base_path, 'stores.csv'))
df_stores

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4
5,6,Quito,Pichincha,D,13
6,7,Quito,Pichincha,D,8
7,8,Quito,Pichincha,D,8
8,9,Quito,Pichincha,B,6
9,10,Quito,Pichincha,C,15


Plot the number of stores per city

In [30]:
grouped = df_stores.groupby('city').agg(count=('store_nbr', 'count')).reset_index()
fig = px.bar(grouped, x='city', y='count', title='Number of store for each city')
fig.show()

Plot the number of stores per state

In [31]:
grouped = df_stores.groupby('state').agg(count=('store_nbr', 'count')).reset_index()
fig = px.bar(grouped, x='state', y='count', title='Number of store for each state')
fig.show()

Join the store metadata df with train df

IMPORTANT: make a left join (between df with the transactions and df stores)

In [33]:
df_train_with_transactions_store_meta = pd.merge(df_train_with_transactions, df_stores, on=['store_nbr'], how='left')
df_train_with_transactions_store_meta

,date,store_nbr,transactions,id,family,sales,onpromotion,year,city,state,type,cluster
0,2013-01-01,1,0.0,0,AUTOMOTIVE,0.000,0,2013,Quito,Pichincha,D,13
1,2013-01-01,1,0.0,1,BABY CARE,0.000,0,2013,Quito,Pichincha,D,13
2,2013-01-01,1,0.0,2,BEAUTY,0.000,0,2013,Quito,Pichincha,D,13
3,2013-01-01,1,0.0,3,BEVERAGES,0.000,0,2013,Quito,Pichincha,D,13
4,2013-01-01,1,0.0,4,BOOKS,0.000,0,2013,Quito,Pichincha,D,13
...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,2017-08-15,9,2155.0,3000883,POULTRY,438.133,0,2017,Quito,Pichincha,B,6
3000884,2017-08-15,9,2155.0,3000884,PREPARED FOODS,154.553,1,2017,Quito,Pichincha,B,6
3000885,2017-08-15,9,2155.0,3000885,PRODUCE,2419.729,148,2017,Quito,Pichincha,B,6
3000886,2017-08-15,9,2155.0,3000886,SCHOOL AND OFFICE SUPPLIES,121.000,8,2017,Quito,Pichincha,B,6


In [34]:
del df_train_with_transactions
del df_stores

Load holidays_events.csv

In [35]:
df_holidays_events = pd.read_csv(os.path.join(base_path, 'holidays_events.csv'))
df_holidays_events

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


In [36]:
df_holidays_events[df_holidays_events['transferred'] == True]

,date,type,locale,locale_name,description,transferred
19,2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
72,2013-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
135,2014-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
255,2016-05-24,Holiday,National,Ecuador,Batalla de Pichincha,True
266,2016-07-25,Holiday,Local,Guayaquil,Fundacion de Guayaquil,True
268,2016-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True
297,2017-01-01,Holiday,National,Ecuador,Primer dia del ano,True
303,2017-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,True
312,2017-05-24,Holiday,National,Ecuador,Batalla de Pichincha,True
324,2017-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True


Join this dataset with train df

Make a left join on the day to include in the data the features from the holidays df
- type
- locale
- locale_name
- description
- transferred

Rename those columns to keep the information that it is related to the holidays

In [37]:
df_holidays_events['date'] = pd.to_datetime(df_holidays_events['date'])

In [38]:
df_holidays_events_renamed = df_holidays_events.rename(
    columns={
        'type': 'holiday_type',
        'locale': 'holiday_locale',
        'locale_name': 'holiday_locale_name',
        'description': 'holiday_description',
        'transferred': 'holiday_transferred'
    }
)

In [39]:
df_holidays_events_renamed[df_holidays_events_renamed['date'] == '2017-08-15']

,date,holiday_type,holiday_locale,holiday_locale_name,holiday_description,holiday_transferred
326,2017-08-15,Holiday,Local,Riobamba,Fundacion de Riobamba,False


In [40]:
df_train_transactions_store_meta_holidays = pd.merge(df_train_with_transactions_store_meta, df_holidays_events_renamed, on=['date'], how='left')
df_train_transactions_store_meta_holidays

,date,store_nbr,transactions,id,family,sales,onpromotion,year,city,state,type,cluster,holiday_type,holiday_locale,holiday_locale_name,holiday_description,holiday_transferred
0,2013-01-01,1,0.0,0,AUTOMOTIVE,0.000,0,2013,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
1,2013-01-01,1,0.0,1,BABY CARE,0.000,0,2013,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
2,2013-01-01,1,0.0,2,BEAUTY,0.000,0,2013,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
3,2013-01-01,1,0.0,3,BEVERAGES,0.000,0,2013,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
4,2013-01-01,1,0.0,4,BOOKS,0.000,0,2013,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,2017-08-15,9,2155.0,3000883,POULTRY,438.133,0,2017,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054344,2017-08-15,9,2155.0,3000884,PREPARED FOODS,154.553,1,2017,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054345,2017-08-15,9,2155.0,3000885,PRODUCE,2419.729,148,2017,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054346,2017-08-15,9,2155.0,3000886,SCHOOL AND OFFICE SUPPLIES,121.000,8,2017,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False


In [41]:
del df_train_with_transactions_store_meta
del df_holidays_events_renamed

Load oil.csv

In [42]:
df_oil = pd.read_csv(os.path.join(base_path, 'oil.csv'))
df_oil

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


Plot the historical oil price

In [43]:
fig = px.line(df_oil, x='date', y='dcoilwtico', title=f'Historical oil price')
fig.show()

In [44]:
df_oil['date'] = pd.to_datetime(df_oil['date'])

In [45]:
df_train_with_oil = pd.merge(df_train_transactions_store_meta_holidays, df_oil, on=['date'], how='left')
df_train_with_oil

,date,store_nbr,transactions,id,family,sales,onpromotion,year,city,state,type,cluster,holiday_type,holiday_locale,holiday_locale_name,holiday_description,holiday_transferred,dcoilwtico
0,2013-01-01,1,0.0,0,AUTOMOTIVE,0.000,0,2013,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,NaN
1,2013-01-01,1,0.0,1,BABY CARE,0.000,0,2013,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,NaN
2,2013-01-01,1,0.0,2,BEAUTY,0.000,0,2013,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,NaN
3,2013-01-01,1,0.0,3,BEVERAGES,0.000,0,2013,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,NaN
4,2013-01-01,1,0.0,4,BOOKS,0.000,0,2013,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,2017-08-15,9,2155.0,3000883,POULTRY,438.133,0,2017,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57
3054344,2017-08-15,9,2155.0,3000884,PREPARED FOODS,154.553,1,2017,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57
3054345,2017-08-15,9,2155.0,3000885,PRODUCE,2419.729,148,2017,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57
3054346,2017-08-15,9,2155.0,3000886,SCHOOL AND OFFICE SUPPLIES,121.000,8,2017,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57


Call pandas bfill() function to remove Nan from the 'dcoilwtico' column

In [46]:
df_train_with_oil['dcoilwtico'] = df_train_with_oil['dcoilwtico'].bfill()

In [47]:
del df_train_transactions_store_meta_holidays
del df_oil

In [48]:
df_train_with_oil

,date,store_nbr,transactions,id,family,sales,onpromotion,year,city,state,type,cluster,holiday_type,holiday_locale,holiday_locale_name,holiday_description,holiday_transferred,dcoilwtico
0,2013-01-01,1,0.0,0,AUTOMOTIVE,0.000,0,2013,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,93.14
1,2013-01-01,1,0.0,1,BABY CARE,0.000,0,2013,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,93.14
2,2013-01-01,1,0.0,2,BEAUTY,0.000,0,2013,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,93.14
3,2013-01-01,1,0.0,3,BEVERAGES,0.000,0,2013,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,93.14
4,2013-01-01,1,0.0,4,BOOKS,0.000,0,2013,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,93.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,2017-08-15,9,2155.0,3000883,POULTRY,438.133,0,2017,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57
3054344,2017-08-15,9,2155.0,3000884,PREPARED FOODS,154.553,1,2017,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57
3054345,2017-08-15,9,2155.0,3000885,PRODUCE,2419.729,148,2017,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57
3054346,2017-08-15,9,2155.0,3000886,SCHOOL AND OFFICE SUPPLIES,121.000,8,2017,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False,47.57


Load all the other dfs

ALL_COMMODITIES_data.csv

COTTON_data.csv oil.csv

ALUMINUM_data.csv

COPPER_data.csv

NATURAL_GAS_data.csv

SUGAR_data.csv

For each df plot the historical values

`Filter the dates we want to keep between 2013-01-01 and 2017-08-31`

And then join with our dataset, BE CAREFUL for those df we have only one data point per month

In [49]:
def load_commodities(filename: str):
    # Load the data
    df = pd.read_csv(os.path.join(base_path, filename))

    df = df[(df['date'] >= '2013-01-01') & (df['date'] <= '2017-08-31')]
    df['date'] = pd.to_datetime(df['date'])
    df['value'] = df['value'].astype(float)
    return df


def read_commodities_and_merge(df_data: pd.DataFrame, filename: str):
    df = load_commodities(filename)

    new_col_name = filename.split('.')[0].lower()
    df = df.rename(
        columns={
            'value': new_col_name,
        }
    )

    df_merged = pd.merge(df_data, df, on='date', how='left')
    df_merged[new_col_name].iloc[-1] = df[new_col_name].iloc[0]
    df_merged[new_col_name] = df_merged[new_col_name].ffill()

    return df_merged

In [50]:
df = load_commodities('ALL_COMMODITIES_data.csv')
fig = px.line(df, x='date', y='value', title=f'Historical all commodities value')
fig.show()

In [51]:
df = load_commodities('ALUMINUM_data.csv')
fig = px.line(df, x='date', y='value', title=f'Historical aluminium value')
fig.show()

In [52]:
df = load_commodities('COPPER_data.csv')
fig = px.line(df, x='date', y='value', title=f'Historical copper value')
fig.show()

In [53]:
df = load_commodities('COTTON_data.csv')
fig = px.line(df, x='date', y='value', title=f'Historical cotton value')
fig.show()

In [54]:
df = load_commodities('NATURAL_GAS_data.csv')
fig = px.line(df, x='date', y='value', title=f'Historical natural gaz value')
fig.show()

In [55]:
df = load_commodities('SUGAR_data.csv')
fig = px.line(df, x='date', y='value', title=f'Historical sugar value')
fig.show()

In [56]:
!ls {base_path}

ALL_COMMODITIES_data.csv  COTTON_data.csv      NATURAL_GAS_data.csv  SUGAR_data.csv
ALUMINUM_data.csv	  data.csv	       oil.csv		     transactions.csv
COPPER_data.csv		  holidays_events.csv  stores.csv


In [57]:
df_with_commodities = read_commodities_and_merge(df_train_with_oil, 'ALL_COMMODITIES_data.csv')
df_with_commodities = read_commodities_and_merge(df_with_commodities, 'COTTON_data.csv')
df_with_commodities = read_commodities_and_merge(df_with_commodities, 'ALUMINUM_data.csv')
df_with_commodities = read_commodities_and_merge(df_with_commodities, 'COPPER_data.csv')
df_with_commodities = read_commodities_and_merge(df_with_commodities, 'NATURAL_GAS_data.csv')
df_with_commodities = read_commodities_and_merge(df_with_commodities, 'SUGAR_data.csv')

<ipython-input-49-ad54742111fb>:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-49-ad54742111fb>:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-49-ad54742111fb>:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-49-ad54742111fb>:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

In [59]:
df_with_commodities.columns

Index(['date', 'store_nbr', 'transactions', 'id', 'family', 'sales',
       'onpromotion', 'year', 'city', 'state', 'type', 'cluster',
       'holiday_type', 'holiday_locale', 'holiday_locale_name',
       'holiday_description', 'holiday_transferred', 'dcoilwtico',
       'all_commodities_data', 'cotton_data', 'aluminum_data', 'copper_data',
       'natural_gas_data', 'sugar_data'],
      dtype='object')

In [60]:
del df_train_with_oil

# 2. Features processing

Add temporal features like:

- is_weekend: 1 if the day is the weekend 0 otherwise

- is_holidays: 1 if the day is holdays 0 otherwise

- month_number: Number of the month (like 12 for december)

- day_week_number: Number of the week day (like 7 for sunday)

- [OPT] is_few_days_before_christmas: 1 if is between 19 and 24 december, 0 otherwise (this might be a cool feature)

In [61]:
df_with_commodities['is_weekend'] = df_with_commodities['date'].dt.dayofweek.isin([5, 6]).astype(int)
df_with_commodities['is_holidays'] = (~df_with_commodities['holiday_description'].isna()).astype(int)
df_with_commodities['month_number'] = df_with_commodities['date'].dt.month
df_with_commodities['day_week_number'] = df_with_commodities['date'].dt.dayofweek + 1

In [62]:
df_with_commodities['is_few_days_before_christmas'] = (
    (df_with_commodities['date'].dt.month == 11) & (df_with_commodities['date'].dt.day >= 19) & (df_with_commodities['date'].dt.day < 25)
).astype(int)

In [63]:
df_with_commodities.columns

Index(['date', 'store_nbr', 'transactions', 'id', 'family', 'sales',
       'onpromotion', 'year', 'city', 'state', 'type', 'cluster',
       'holiday_type', 'holiday_locale', 'holiday_locale_name',
       'holiday_description', 'holiday_transferred', 'dcoilwtico',
       'all_commodities_data', 'cotton_data', 'aluminum_data', 'copper_data',
       'natural_gas_data', 'sugar_data', 'is_weekend', 'is_holidays',
       'month_number', 'day_week_number', 'is_few_days_before_christmas'],
      dtype='object')

Identify the categorical features

In [64]:
df_with_commodities.columns

Index(['date', 'store_nbr', 'transactions', 'id', 'family', 'sales',
       'onpromotion', 'year', 'city', 'state', 'type', 'cluster',
       'holiday_type', 'holiday_locale', 'holiday_locale_name',
       'holiday_description', 'holiday_transferred', 'dcoilwtico',
       'all_commodities_data', 'cotton_data', 'aluminum_data', 'copper_data',
       'natural_gas_data', 'sugar_data', 'is_weekend', 'is_holidays',
       'month_number', 'day_week_number', 'is_few_days_before_christmas'],
      dtype='object')

For each categorical features display the number of modalities

In [66]:
len(df_with_commodities['store_nbr'].unique())

54

In [67]:
len(df_with_commodities['family'].unique())

33

In [68]:
len(df_with_commodities['holiday_type'].unique())

7

In [69]:
len(df_with_commodities['holiday_locale_name'].unique())

25

In [ ]:
len(df_with_commodities['holiday_locale_name'].unique())

In [70]:
df_with_commodities = df_with_commodities[::-1]

In [71]:
df_with_commodities

,date,store_nbr,transactions,id,family,sales,onpromotion,year,city,state,...,cotton_data,aluminum_data,copper_data,natural_gas_data,sugar_data,is_weekend,is_holidays,month_number,day_week_number,is_few_days_before_christmas
3054347,2017-08-15,9,2155.0,3000887,SEAFOOD,16.000,0,2017,Quito,Pichincha,...,79.340909,2030.011364,6485.625000,2.90,13.796087,0,1,8,2,0
3054346,2017-08-15,9,2155.0,3000886,SCHOOL AND OFFICE SUPPLIES,121.000,8,2017,Quito,Pichincha,...,79.340909,2030.011364,6485.625000,2.90,13.796087,0,1,8,2,0
3054345,2017-08-15,9,2155.0,3000885,PRODUCE,2419.729,148,2017,Quito,Pichincha,...,79.340909,2030.011364,6485.625000,2.90,13.796087,0,1,8,2,0
3054344,2017-08-15,9,2155.0,3000884,PREPARED FOODS,154.553,1,2017,Quito,Pichincha,...,79.340909,2030.011364,6485.625000,2.90,13.796087,0,1,8,2,0
3054343,2017-08-15,9,2155.0,3000883,POULTRY,438.133,0,2017,Quito,Pichincha,...,79.340909,2030.011364,6485.625000,2.90,13.796087,0,1,8,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2013-01-01,1,0.0,4,BOOKS,0.000,0,2013,Quito,Pichincha,...,85.506818,2037.606818,8053.738636,3.33,18.845714,0,1,1,2,0
3,2013-01-01,1,0.0,3,BEVERAGES,0.000,0,2013,Quito,Pichincha,...,85.506818,2037.606818,8053.738636,3.33,18.845714,0,1,1,2,0
2,2013-01-01,1,0.0,2,BEAUTY,0.000,0,2013,Quito,Pichincha,...,85.506818,2037.606818,8053.738636,3.33,18.845714,0,1,1,2,0
1,2013-01-01,1,0.0,1,BABY CARE,0.000,0,2013,Quito,Pichincha,...,85.506818,2037.606818,8053.738636,3.33,18.845714,0,1,1,2,0


Remove the columns we don't need like:

- id

- year (if you have it)

- holiday_description

In [72]:
df_with_commodities = df_with_commodities.drop(['id', 'year', 'holiday_description'], axis=1)

In [73]:
df_with_commodities.columns

Index(['date', 'store_nbr', 'transactions', 'family', 'sales', 'onpromotion',
       'city', 'state', 'type', 'cluster', 'holiday_type', 'holiday_locale',
       'holiday_locale_name', 'holiday_transferred', 'dcoilwtico',
       'all_commodities_data', 'cotton_data', 'aluminum_data', 'copper_data',
       'natural_gas_data', 'sugar_data', 'is_weekend', 'is_holidays',
       'month_number', 'day_week_number', 'is_few_days_before_christmas'],
      dtype='object')

Convert holiday_transferred feature to int (do not forget to fillna(0) on this column)

In [74]:
df_with_commodities['holiday_transferred'] = df_with_commodities['holiday_transferred'].fillna(0).astype(int)

In [75]:
# Categorical features
categorical_features = [
    'store_nbr',
    'family',
    'city',
    'state',
    'type',
    'cluster',
    'holiday_type',
    'holiday_locale',
    'holiday_locale_name',
    'month_number',
    'day_week_number'
]

# Numerical features

# We can use historical (1 day shift)
numerical_features_labels = [
    'transactions',
    'sales',
    'dcoilwtico',
]

# We can use current feature
numerical_features = [
    'onpromotion',
    'holiday_transferred',
    'is_weekend',
    'is_holidays',
    'is_few_days_before_christmas',
    'all_commodities_data',
    'cotton_data',
    'aluminum_data',
    'copper_data',
    'natural_gas_data',
    'sugar_data',
]

Save the data into parquet to keep track of the previous work

In [81]:
df_with_commodities.to_parquet('df_data.parquet')

# 3. Feature engineering

Here define the proper features for our model,

some features can be used at any time of the time serie

In [82]:
df_data = pd.read_parquet('df_data.parquet')

In [85]:
df_data.columns

Index(['date', 'store_nbr', 'transactions', 'family', 'sales', 'onpromotion',
       'city', 'state', 'type', 'cluster', 'holiday_type', 'holiday_locale',
       'holiday_locale_name', 'holiday_transferred', 'dcoilwtico',
       'all_commodities_data', 'cotton_data', 'aluminum_data', 'copper_data',
       'natural_gas_data', 'sugar_data', 'is_weekend', 'is_holidays',
       'month_number', 'day_week_number', 'is_few_days_before_christmas'],
      dtype='object')

To make the feature engineering,
we need to think a bit about:
- Which feature is suposed to be predictive ?
- Which feature can be used as historical feature ?
- ...

For instance `transactions` can be used has historical feature

For each historical feature, build a function that is going to compute multiple orther features like:
- Last value
- Second last value
- Third last value
- Variation last 7 days
- Variation last 30 days
- Rolling window last 7 days
- ... (you can add more features)

In [86]:
df_data['date'].iloc[0], df_data['date'].iloc[-1]

(Timestamp('2017-08-15 00:00:00'), Timestamp('2013-01-01 00:00:00'))

In [93]:
def generate_grouped_historical_features(df, historical_features):
    # Ensure 'date' column is in datetime format
    df['date'] = pd.to_datetime(df['date'])
    # Sort DataFrame by 'store_nbr', 'family', and 'date' from oldest to most recent
    df.sort_values(by=['store_nbr', 'family', 'date'], ascending=True, inplace=True)

    # Define a function to apply to each group
    def calculate_features(group):
        for feature in historical_features:
            # Generate the last, second last, and third last values
            group[f'{feature}_last_value'] = group[feature].shift(1)
            group[f'{feature}_second_last_value'] = group[feature].shift(2)
            group[f'{feature}_third_last_value'] = group[feature].shift(3)

            # Generate variation over the last 7 and 30 days
            for lag in [7, 30]:
                group[f'{feature}_variation_last_{lag}_days'] = group[feature] - group[feature].shift(lag)

            # Calculate rolling window of the last 7 days
            group[f'{feature}_rolling_window_last_7_days'] = group[feature].rolling(window=7, min_periods=1).mean()

        return group

    # Apply the function to each group
    df = df.groupby(['store_nbr', 'family']).apply(calculate_features)

    # Reset the index to handle the multi-index created by groupby and apply
    df.reset_index(drop=True, inplace=True)

    return df

In [94]:
historical_features = [
    'transactions', 'sales', 'dcoilwtico', 'all_commodities_data',
    'cotton_data', 'aluminum_data', 'copper_data',
    'natural_gas_data', 'sugar_data'
]

df_data = generate_grouped_historical_features(df_data, historical_features)

<ipython-input-93-1eaa7e6dfba5>:25: FutureWarning:

Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)



In [97]:
df_data.columns

Index(['date', 'store_nbr', 'transactions', 'family', 'sales', 'onpromotion',
       'city', 'state', 'type', 'cluster', 'holiday_type', 'holiday_locale',
       'holiday_locale_name', 'holiday_transferred', 'dcoilwtico',
       'all_commodities_data', 'cotton_data', 'aluminum_data', 'copper_data',
       'natural_gas_data', 'sugar_data', 'is_weekend', 'is_holidays',
       'month_number', 'day_week_number', 'is_few_days_before_christmas',
       'transactions_last_value', 'transactions_second_last_value',
       'transactions_third_last_value', 'transactions_variation_last_7_days',
       'transactions_variation_last_30_days',
       'transactions_rolling_window_last_7_days', 'sales_last_value',
       'sales_second_last_value', 'sales_third_last_value',
       'sales_variation_last_7_days', 'sales_variation_last_30_days',
       'sales_rolling_window_last_7_days', 'dcoilwtico_last_value',
       'dcoilwtico_second_last_value', 'dcoilwtico_third_last_value',
       'dcoilwtico_variati

In [100]:
df_data[(df_data['store_nbr'] == 2) & (df_data['family'] == 'AUTOMOTIVE')][['date', 'transactions', 'transactions_last_value', 'transactions_third_last_value']]

,date,transactions,transactions_last_value,transactions_third_last_value
56562,2013-01-01,0.0,NaN,NaN
56563,2013-01-02,2358.0,0.0,NaN
56564,2013-01-03,2033.0,2358.0,NaN
56565,2013-01-04,2066.0,2033.0,0.0
56566,2013-01-05,2062.0,2066.0,2358.0
...,...,...,...,...
58271,2017-08-11,1698.0,1695.0,1688.0
58272,2017-08-12,1613.0,1698.0,1786.0
58273,2017-08-13,1658.0,1613.0,1695.0
58274,2017-08-14,1788.0,1658.0,1698.0


Now extract you labels and your features

do not forget to remove all your historical features

In [101]:
y = df_data['sales']

In [104]:
df_data = df_data.drop(historical_features + ['date'], axis=1)
df_data.columns

Index(['store_nbr', 'family', 'onpromotion', 'city', 'state', 'type',
       'cluster', 'holiday_type', 'holiday_locale', 'holiday_locale_name',
       'holiday_transferred', 'is_weekend', 'is_holidays', 'month_number',
       'day_week_number', 'is_few_days_before_christmas',
       'transactions_last_value', 'transactions_second_last_value',
       'transactions_third_last_value', 'transactions_variation_last_7_days',
       'transactions_variation_last_30_days',
       'transactions_rolling_window_last_7_days', 'sales_last_value',
       'sales_second_last_value', 'sales_third_last_value',
       'sales_variation_last_7_days', 'sales_variation_last_30_days',
       'sales_rolling_window_last_7_days', 'dcoilwtico_last_value',
       'dcoilwtico_second_last_value', 'dcoilwtico_third_last_value',
       'dcoilwtico_variation_last_7_days', 'dcoilwtico_variation_last_30_days',
       'dcoilwtico_rolling_window_last_7_days',
       'all_commodities_data_last_value',
       'all_commoditie

Build a list with your categorical features we will need this list to build our catboost classifier

In [105]:
categorical_features = [
    'store_nbr', 'family', 'onpromotion', 'city',
    'state', 'type', 'cluster', 'holiday_type',
    'holiday_locale', 'holiday_locale_name', 'holiday_transferred',
    'month_number', 'day_week_number'
]

Fill nan values with 0s

In [108]:
df_data = df_data.fillna(0)

Separate your dataset between train and test

In [106]:
from sklearn.model_selection import train_test_split

In [109]:
X_train, X_test, y_train, y_test = train_test_split(df_data, y, test_size=0.33, random_state=42)

In [110]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2046413, 70), (1007935, 70), (2046413,), (1007935,))

# 3. Build models

In [112]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 16.2 MB/s eta 0:00:00


In [127]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [117]:
model = CatBoostRegressor(
    iterations=200,
    learning_rate=0.01,
    depth=6,
    cat_features=categorical_features
)

In [118]:
model.fit(X_train, y_train, verbose=True)

0:	learn: 1101.7230122	total: 1.38s	remaining: 4m 34s
1:	learn: 1091.9986728	total: 2.68s	remaining: 4m 25s
2:	learn: 1082.3727270	total: 3.87s	remaining: 4m 13s
3:	learn: 1072.8514581	total: 5.04s	remaining: 4m 6s
4:	learn: 1063.4095304	total: 6.35s	remaining: 4m 7s
5:	learn: 1054.1201232	total: 7.54s	remaining: 4m 3s
6:	learn: 1044.8984407	total: 8.67s	remaining: 3m 58s
7:	learn: 1035.8149178	total: 9.88s	remaining: 3m 57s
8:	learn: 1026.7894324	total: 11.1s	remaining: 3m 54s
9:	learn: 1017.8591933	total: 12.2s	remaining: 3m 52s
10:	learn: 1009.0198097	total: 13.5s	remaining: 3m 51s
11:	learn: 1000.2316158	total: 14.7s	remaining: 3m 50s
12:	learn: 991.5312738	total: 15.7s	remaining: 3m 45s
13:	learn: 982.9905332	total: 17s	remaining: 3m 45s
14:	learn: 974.5280296	total: 18.3s	remaining: 3m 45s
15:	learn: 966.1244310	total: 19.6s	remaining: 3m 45s
16:	learn: 957.8066411	total: 20.7s	remaining: 3m 43s
17:	learn: 949.5997060	total: 22s	remaining: 3m 42s
18:	learn: 941.4824512	total: 23.

In [120]:
predictions

array([70.5618379 , 74.74199867, 70.5618379 , ..., 70.5618379 ,
       70.5618379 , 70.78448468])

In [121]:
y_test

668753        0.0
1267004      25.0
997442        3.0
873596        1.0
756145     2186.0
            ...  
2023684     234.0
188151      111.0
2300015       7.0
1816924       0.0
312295        2.0
Name: sales, Length: 1007935, dtype: float64

In [119]:
predictions = model.predict(X_test)

# Calculate the MSE
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 74695.83170697723


In [130]:
preds = model.predict(X_test)
preds

array([70.5618379 , 74.74199867, 70.5618379 , ..., 70.5618379 ,
       70.5618379 , 70.78448468])

In [123]:
mean_absolute_error(y_test, predictions)

105.18290912474251

In [134]:
import numpy as np

def compute_rmse(y_true, y_pred):
    difference = y_true - y_pred
    squared_difference = np.square(difference)
    mean_squared_difference = np.mean(squared_difference)
    rmse = np.sqrt(mean_squared_difference)
    return rmse

In [135]:
compute_rmse(y_test, predictions)

273.30538177463177